In [1]:
!pip3 install langchain_community langchain-ollama


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM

In [7]:
llm = OllamaLLM(model="gemma3")
llm.invoke("who is the current president of Brazil?")

'As of today, November 2, 2023, the current president of Brazil is **Luiz Inácio Lula da Silva**, often referred to as simply **Lula**. \n\nHe assumed office on January 1, 2023.'

In [10]:
import pandas as pd
df = pd.read_csv("C:/Users/Bruno/Downloads/Nubank_2025-06-19.csv")
df.head()

,date,title,amount
0,2025-06-11,Uber* Trip,9.74
1,2025-06-10,99app *99app,12.40
2,2025-06-09,Uber Uber *Trip Help.U,31.42
3,2025-06-08,Uber* Trip,8.46
4,2025-06-08,Uber Uber *Trip Help.U,15.22


In [6]:
if 'Identificador' in df.columns:
                df.drop(columns=['Identificador'], inplace=True)

df.head()

,Data,Valor,Descrição
0,01/04/2025,500.00,Transferência recebida pelo Pix - MARGARETE MA...
1,02/04/2025,-60.00,Transferência enviada pelo Pix - Jéssica Diana...
2,03/04/2025,-131.30,Transferência enviada pelo Pix - COMPANHIA PIR...
3,03/04/2025,-55.18,Pagamento de boleto efetuado - SERVICO AUTONOM...
4,06/04/2025,-7.00,Compra no débito - Meep Pa*Multi Produtor


In [11]:
# Get unique transactions in the Name / Description column
unique_transactions = df["title"].unique()
len(unique_transactions)

21

In [12]:
unique_transactions[1:30]

array(['99app *99app', 'Uber Uber *Trip Help.U',
       'Crédito de parcelamento de compra', 'Sonda Salto Ii',
       'Dia Brasil Lj', 'Dm*Crunchyroll L', 'Ppro *Microsoft', 'Adega 4d',
       'Gustavo Moura de Sa', 'Gustavo Mattos Fortes',
       'Parcelamento de Compra - Ppro *Microsoft - 1/2',
       'Parcelamento de Compra - Nove de Julho Conven - 1/3',
       'Nove de Julho Conven', 'Blablacar', 'Prohumana',
       'Parcelamento de Compra - Sonda Salto Ii - 2/3',
       'Parcelamento de Compra - Sonda Salto Ii - 3/3',
       'Parcelamento de Compra - Microsoft*Subscription - 2/2',
       'Parcelamento de Compra - Fatimo Volmir Oliveir - 7/7',
       'Pagamento recebido'], dtype=object)

In [13]:
def clean_transaction_name(transaction_names):
    """
    Remove informações desnecessárias da transação após o segundo hífen
    """
    # Divide a string pelos hífens
    parts = transaction_names.split(' - ')

    # Se tem mais de 2 partes, mantém apenas as duas primeiras
    if len(parts) > 2:
        return ' - '.join(parts[:2])

    return transaction_names

def categorize_transactions(transaction_names, llm):
    individual_transactions = [t.strip() for t in transaction_names.split(',')]

    cleaned_transactions = [clean_transaction_name(t) for t in individual_transactions]

    # Cria uma lista numerada para facilitar o processamento
    numbered_transactions = []
    for i, transaction in enumerate(cleaned_transactions, 1):
        numbered_transactions.append(f"{i}. {transaction}")
    
    transactions_text = '\n'.join(numbered_transactions)
    
    prompt = (
        "Você é um especialista em contexto de conjunto de dados financeiros. "
        "Seu papel é analisar e classificar receitas e despesas do contexto econômico do indivíduo. "
        
        "REGRAS IMPORTANTES:\n"
        "- Retorne EXATAMENTE no formato 'TRANSAÇÃO - CATEGORIA' (uma por linha)\n"
        "- Mantenha a transação EXATAMENTE como foi enviada, não abrevie nem corte\n"
        "- Quando identificar supermercados/mercados, classificar EXATAMENTE como 'Mercado'\n"
        "- Quando identificar alimentações, refeições, restaurantes e pedidos de comida (via Ifood por exemplo), classificar EXATAMENTE como 'Alimentação'\n"
        "- Quando identificar farmácia ou drogaria, classificar EXATAMENTE como 'Saúde'\n"
        "- NÃO adicione numeração na resposta\n"
        "- NÃO adicione explicações, apenas a classificação\n"
        "- Processe TODAS as transações enviadas\n\n"
        
        "Categorias disponíveis: Alimentação, Transporte, Saúde, Mercado, Educação, Lazer,"
        "Moradia, Investimentos, Streaming, Transferências, Outros\n\n"
        
        "Transações para categorizar:\n" + transactions_text
    )

    response = llm.invoke(prompt)
    
    response_lines = [line.strip() for line in response.split('\n') if line.strip()]

    print("LLM Response:")
    print(response_lines)

    # Process each line and handle different formats
    transactions = []
    categories = []

    for line in response_lines:
        if ' - ' in line:
            # Split only on the last occurrence of ' - ' para pegar a categoria
            parts = line.rsplit(' - ', 1)
            if len(parts) == 2:
                transaction = parts[0].strip()
                category = parts[1].strip()
                
                # Remove numeração se ainda existir
                if '. ' in transaction and transaction.split('. ')[0].isdigit():
                    transaction = '. '.join(transaction.split('. ')[1:])
                
                transactions.append(transaction)
                categories.append(category)
            else:
                # Fallback for malformed lines
                transactions.append(line.strip())
                categories.append("Não categorizado")
        else:
            # Handle lines without separator
            if line.strip() and not line.strip().startswith('TRANSAÇÃO'):
                transactions.append(line.strip())
                categories.append("Não categorizado")

    # Validação: verifica se todas as transações foram processadas
    if len(transactions) != len(cleaned_transactions):
        print(f"AVISO: Esperado {len(cleaned_transactions)} transações, mas obteve {len(transactions)}")
        print(f"Transações originais: {cleaned_transactions}")
        print(f"Transações processadas: {transactions}")

    # Create DataFrame
    categories_df = pd.DataFrame({
        'Transaction': transactions,
        'Category': categories
    })

    # Remove empty rows if any
    categories_df = categories_df[categories_df['Transaction'] != '']

    return categories_df

In [16]:
categorize_transactions('Uber Uber *Trip Help.U, Parcelamento de Compra - Ppro *Microsoft - 1/2, Pagamento de boleto efetuado - SERVICO AUTONOMO DE AGUA E ESGOTO DE SALTO', llm=llm)

LLM Response:
['Uber - Transporte', 'Parcelamento de Compra - Ppro - Outros', 'Pagamento de boleto efetuado - SERVICO AUTONOMO DE AGUA E ESGOTO DE SALTO - Moradia']


,Transaction,Category
0,Uber,Transporte
1,Parcelamento de Compra - Ppro,Outros
2,Pagamento de boleto efetuado - SERVICO AUTONOM...,Moradia


In [23]:
import os
import pickle
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage
from tqdm import tqdm

# ======== CONFIGURAÇÕES ========
OLLAMA_MODEL = "gemma3"
CSV_PATH = "C:/Users/Bruno/Downloads/Nubank_2025-06-19.csv"
DESCRICAO_COL = "title"
VALOR_COL = "amount"
CACHE_PATH = "categorias_cache.pkl"
BLOCO_TAMANHO = 10

# ======== CACHE ========
def load_cache(path):
    if os.path.exists(path):
        with open(path, 'rb') as f:
            return pickle.load(f)
    return {}

def save_cache(cache, path):
    with open(path, 'wb') as f:
        pickle.dump(cache, f)

# ======== LIMPEZA E PROMPT ========
def clean_transaction_name(transaction_name):
    """Limpa o nome da transação para padronização"""
    parts = transaction_name.split(' - ')
    return ' - '.join(parts[:2]) if len(parts) > 2 else transaction_name

def generate_prompt(transactions):
    """Gera prompt para categorização em lote"""
    formatted = '\n'.join(f"{i+1}. {clean_transaction_name(t)}" for i, t in enumerate(transactions))
    prompt = f"""
Você é um especialista em análise de finanças pessoais.

REGRAS IMPORTANTES:
- Retorne EXATAMENTE no formato: 'TRANSAÇÃO - CATEGORIA'
- Use os nomes das transações EXATAMENTE como fornecidos (após limpeza)
- Categorias possíveis: Alimentação, Transporte, Saúde, Mercado, Educação, Lazer, Moradia, Investimentos, Streaming, Transferências, Outros
- Não adicione numeração, explicações ou comentários extras
- Categorize TODAS as {len(transactions)} transações listadas abaixo:

Transações:
{formatted}

Exemplo de formato esperado:
Uber* Trip - Transporte
Netflix - Streaming
Supermercado XYZ - Mercado
"""
    return prompt.strip()

def parse_llm_response(response, original_transactions):
    """Parse da resposta do LLM com validação"""
    lines = [line.strip() for line in response.split('\n') if line.strip()]
    parsed = []
    
    print(f"🔍 Resposta do LLM ({len(lines)} linhas):")
    for line in lines:
        print(f"  -> {line}")
    
    # Tentar diferentes formatos de parsing
    for line in lines:
        if ' - ' in line:
            # Formato padrão: "Transação - Categoria"
            trans, cat = line.rsplit(' - ', 1)
            parsed.append((trans.strip(), cat.strip()))
        elif ':' in line and not line.startswith(('Exemplo', 'Categorias')):
            # Formato alternativo: "Transação: Categoria"
            trans, cat = line.split(':', 1)
            parsed.append((trans.strip(), cat.strip()))
    
    print(f"✅ Parsed {len(parsed)} categorias")
    return parsed

# ======== CATEGORIZAÇÃO COM CACHE E BLOCOS ========
def categorize_transactions_in_blocks(df, model_name=OLLAMA_MODEL, bloco_tamanho=BLOCO_TAMANHO):
    """Categoriza transações usando cache e processamento em blocos"""
    
    # Filtrar apenas despesas (valores positivos no seu caso)
    df_despesas = df[df[VALOR_COL] > 0].copy()
    transacoes = df_despesas[DESCRICAO_COL].tolist()
    
    print(f"📊 Total de despesas encontradas: {len(transacoes)}")
    
    cache = load_cache(CACHE_PATH)
    print(f"📁 Cache carregado com {len(cache)} entradas")
    
    llm = ChatOllama(model=model_name)
    
    # Verificar quais transações precisam ser categorizadas
    transacoes_para_categorizar = []
    indices_para_categorizar = []
    
    for idx, t in enumerate(transacoes):
        t_clean = clean_transaction_name(t)
        if t_clean not in cache:
            transacoes_para_categorizar.append(t)
            indices_para_categorizar.append(idx)
    
    print(f"🔄 {len(transacoes_para_categorizar)} transações novas para categorizar com LLM.")
    
    # Processar transações não cachadas por blocos
    if transacoes_para_categorizar:
        for i in tqdm(range(0, len(transacoes_para_categorizar), bloco_tamanho), desc="Categorizando"):
            bloco = transacoes_para_categorizar[i:i+bloco_tamanho]
            prompt = generate_prompt(bloco)
            
            try:
                print(f"\n🤖 Enviando bloco de {len(bloco)} transações para o LLM...")
                resposta = llm.invoke([HumanMessage(content=prompt)])
                resultado = parse_llm_response(resposta.content, bloco)
                
                # Salvar no cache
                for trans, cat in resultado:
                    t_clean = clean_transaction_name(trans)
                    cache[t_clean] = cat
                    print(f"  ✅ {t_clean} -> {cat}")
                
            except Exception as e:
                print(f"❌ Erro ao processar bloco: {e}")
                # Em caso de erro, categorizar como "Outros"
                for trans in bloco:
                    t_clean = clean_transaction_name(trans)
                    cache[t_clean] = "Outros"
    
    # Salvar cache atualizado
    save_cache(cache, CACHE_PATH)
    print(f"💾 Cache salvo com {len(cache)} entradas")
    
    # Aplicar categorias a todas as transações
    categorias_finais = []
    for t in transacoes:
        t_clean = clean_transaction_name(t)
        categoria = cache.get(t_clean, "Não categorizado")
        categorias_finais.append(categoria)
    
    df_despesas['Categoria'] = categorias_finais
    
    # Debug: mostrar resultado
    print("\n📋 Resumo das categorias:")
    print(df_despesas['Categoria'].value_counts())
    
    return df_despesas

# ======== EXECUÇÃO PRINCIPAL ========
def main():
    """Função principal"""
    print("🚀 Iniciando categorização de transações...")
    
    # Carregar dados
    try:
        df = pd.read_csv(CSV_PATH)
        print(f"📄 Arquivo carregado: {len(df)} transações")
    except FileNotFoundError:
        print(f"❌ Arquivo não encontrado: {CSV_PATH}")
        return
    except Exception as e:
        print(f"❌ Erro ao carregar arquivo: {e}")
        return
    
    # Remover coluna Identificador se existir
    if 'Identificador' in df.columns:
        df.drop(columns=['Identificador'], inplace=True)
        print("🗑️ Coluna 'Identificador' removida")
    
    # Verificar se as colunas necessárias existem
    if DESCRICAO_COL not in df.columns or VALOR_COL not in df.columns:
        print(f"❌ Colunas necessárias não encontradas. Esperado: {DESCRICAO_COL}, {VALOR_COL}")
        print(f"Colunas encontradas: {list(df.columns)}")
        return
    
    # Categorizar transações
    df_categorizado = categorize_transactions_in_blocks(df, model_name=OLLAMA_MODEL, bloco_tamanho=BLOCO_TAMANHO)
    
    # Fazer merge com dados originais
    df_final = df.merge(
        df_categorizado[[DESCRICAO_COL, 'Categoria']], 
        on=DESCRICAO_COL, 
        how='left'
    )
    
    # Preencher categorias vazias (receitas/transferências ficam sem categoria ou com categoria específica)
    df_final['Categoria'] = df_final['Categoria'].fillna('')
    
    # Aplicar lógica específica para valores negativos
    mask_negativos = df_final[VALOR_COL] < 0
    df_final.loc[mask_negativos & df_final['Categoria'].isna(), 'Categoria'] = 'Receita'
    
    # Salvar resultado
    output_path = "extrato_categorizados_final.csv"
    df_final.to_csv(output_path, index=False)
    print(f"✅ Arquivo final salvo como '{output_path}'")
    
    # Mostrar estatísticas finais
    print(f"\n📊 Estatísticas finais:")
    print(f"Total de transações: {len(df_final)}")
    print(f"Despesas categorizadas: {len(df_final[df_final[VALOR_COL] > 0])}")
    print(f"Receitas/Créditos: {len(df_final[df_final[VALOR_COL] <= 0])}")
    
    # Mostrar distribuição de categorias
    categorias_count = df_final[df_final['Categoria'] != '']['Categoria'].value_counts()
    print(f"\n🏷️ Distribuição de categorias:")
    for cat, count in categorias_count.items():
        print(f"  {cat}: {count}")

if __name__ == "__main__":
    main()

🚀 Iniciando categorização de transações...
📄 Arquivo carregado: 49 transações
📊 Total de despesas encontradas: 46
📁 Cache carregado com 0 entradas
🔄 46 transações novas para categorizar com LLM.


Categorizando:   0%|          | 0/5 [00:00<?, ?it/s]


🤖 Enviando bloco de 10 transações para o LLM...


Categorizando:  20%|██        | 1/5 [00:03<00:13,  3.30s/it]

🔍 Resposta do LLM (10 linhas):
  -> Uber* Trip - Transporte
  -> 99app *99app - Transporte
  -> Uber Uber *Trip Help.U - Transporte
  -> Uber* Trip - Transporte
  -> Uber Uber *Trip Help.U - Transporte
  -> Uber* Trip - Transporte
  -> Uber Uber *Trip Help.U - Transporte
  -> Uber* Trip - Transporte
  -> 99app *99app - Transporte
  -> Uber Uber *Trip Help.U - Transporte
✅ Parsed 10 categorias
  ✅ Uber* Trip -> Transporte
  ✅ 99app *99app -> Transporte
  ✅ Uber Uber *Trip Help.U -> Transporte
  ✅ Uber* Trip -> Transporte
  ✅ Uber Uber *Trip Help.U -> Transporte
  ✅ Uber* Trip -> Transporte
  ✅ Uber Uber *Trip Help.U -> Transporte
  ✅ Uber* Trip -> Transporte
  ✅ 99app *99app -> Transporte
  ✅ Uber Uber *Trip Help.U -> Transporte

🤖 Enviando bloco de 10 transações para o LLM...


...

🔍 Resposta do LLM (10 linhas):
  -> Uber* Trip - Transporte
  -> Adega 4d - Alimentação
  -> 99app *99app - Transferências
  -> 99app *99app - Transferências
  -> Uber Uber *Trip Help.U - Transporte
  -> Gustavo Moura de Sa - Outros
  -> Gustavo Mattos Fortes - Outros
  -> 99app *99app - Transferências
  -> 99app *99app - Transferências
  -> Parcelamento de Compra - Ppro *Microsoft - Mercado
✅ Parsed 10 categorias
  ✅ Uber* Trip -> Transporte
  ✅ Adega 4d -> Alimentação
  ✅ 99app *99app -> Transferências
  ✅ 99app *99app -> Transferências
  ✅ Uber Uber *Trip Help.U -> Transporte
  ✅ Gustavo Moura de Sa -> Outros
  ✅ Gustavo Mattos Fortes -> Outros
  ✅ 99app *99app -> Transferências
  ✅ 99app *99app -> Transferências
  ✅ Parcelamento de Compra - Ppro *Microsoft -> Mercado

🤖 Enviando bloco de 10 transações para o LLM...


Categorizando:  80%|████████  | 4/5 [00:07<00:01,  1.69s/it]

🔍 Resposta do LLM (10 linhas):
  -> Parcelamento de Compra - Nove de Julho Conven - Outros
  -> Nove de Julho Conven - Outros
  -> Blablacar - Transporte
  -> 99app *99app - Outros
  -> Uber* Trip - Transporte
  -> Uber Uber *Trip Help.U - Transporte
  -> 99app *99app - Outros
  -> Prohumana - Mercado
  -> 99app *99app - Outros
  -> 99app *99app - Outros
✅ Parsed 10 categorias
  ✅ Parcelamento de Compra - Nove de Julho Conven -> Outros
  ✅ Nove de Julho Conven -> Outros
  ✅ Blablacar -> Transporte
  ✅ 99app *99app -> Outros
  ✅ Uber* Trip -> Transporte
  ✅ Uber Uber *Trip Help.U -> Transporte
  ✅ 99app *99app -> Outros
  ✅ Prohumana -> Mercado
  ✅ 99app *99app -> Outros
  ✅ 99app *99app -> Outros

🤖 Enviando bloco de 6 transações para o LLM...


Categorizando: 100%|██████████| 5/5 [00:08<00:00,  1.75s/it]

🔍 Resposta do LLM (6 linhas):
  -> Parcelamento de Compra - Sonda Salto Ii - Mercado
  -> Uber Uber *Trip Help.U - Transporte
  -> Parcelamento de Compra - Sonda Salto Ii - Mercado
  -> Uber* Trip - Transporte
  -> Parcelamento de Compra - Microsoft*Subscription - Educação
  -> Parcelamento de Compra - Fatimo Volmir Oliveir - Outros
✅ Parsed 6 categorias
  ✅ Parcelamento de Compra - Sonda Salto Ii -> Mercado
  ✅ Uber Uber *Trip Help.U -> Transporte
  ✅ Parcelamento de Compra - Sonda Salto Ii -> Mercado
  ✅ Uber* Trip -> Transporte
  ✅ Parcelamento de Compra - Microsoft*Subscription -> Educação
  ✅ Parcelamento de Compra - Fatimo Volmir Oliveir -> Outros
💾 Cache salvo com 18 entradas

📋 Resumo das categorias:
Categoria
Outros         20
Transporte     17
Mercado         4
Lazer           2
Streaming       1
Alimentação     1
Educação        1
Name: count, dtype: int64
✅ Arquivo final salvo como 'extrato_categorizados_final.csv'

📊 Estatísticas finais:
Total de transações: 343
Despesas c

In [24]:
import os
import pickle
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage
from tqdm import tqdm

# ======== CONFIGURAÇÕES ========
OLLAMA_MODEL = "gemma3"
CSV_PATH = "C:/Users/Bruno/Downloads/Nubank_2025-06-19.csv"
DESCRICAO_COL = "title"
VALOR_COL = "amount"
CACHE_PATH = "categorias_cache.pkl"
BLOCO_TAMANHO = 10

# ======== CACHE ========
def load_cache(path):
    if os.path.exists(path):
        with open(path, 'rb') as f:
            return pickle.load(f)
    return {}

def save_cache(cache, path):
    with open(path, 'wb') as f:
        pickle.dump(cache, f)

# ======== LIMPEZA E PROMPT ========
def clean_transaction_name(transaction_name):
    """Limpa o nome da transação para padronização"""
    parts = transaction_name.split(' - ')
    return ' - '.join(parts[:2]) if len(parts) > 2 else transaction_name

def generate_prompt(transactions):
    """Gera prompt para categorização em lote"""
    formatted = '\n'.join(f"{i+1}. {clean_transaction_name(t)}" for i, t in enumerate(transactions))
    prompt = f"""
Você é um especialista em análise de finanças pessoais.

REGRAS IMPORTANTES:
- Retorne EXATAMENTE no formato: 'TRANSAÇÃO - CATEGORIA'
- Use os nomes das transações EXATAMENTE como fornecidos (após limpeza)
- Categorias possíveis: Alimentação, Transporte, Saúde, Mercado, Educação, Lazer, Moradia, Investimentos, Streaming, Transferências, Outros
- Não adicione numeração, explicações ou comentários extras
- Categorize TODAS as {len(transactions)} transações listadas abaixo:

Transações:
{formatted}

Exemplo de formato esperado:
Uber* Trip - Transporte
Netflix - Streaming
Supermercado XYZ - Mercado
"""
    return prompt.strip()

def parse_llm_response(response, original_transactions):
    """Parse da resposta do LLM com validação melhorada"""
    lines = [line.strip() for line in response.split('\n') if line.strip()]
    parsed = []
    
    print(f"🔍 Resposta do LLM ({len(lines)} linhas):")
    for line in lines[:5]:  # Mostrar apenas as primeiras 5 linhas
        print(f"  -> {line}")
    
    # Criar mapeamento das transações originais limpas
    trans_originais = {clean_transaction_name(t): t for t in original_transactions}
    
    # Tentar diferentes formatos de parsing
    for line in lines:
        if ' - ' in line:
            # Formato padrão: "Transação - Categoria"
            trans, cat = line.rsplit(' - ', 1)
            trans_clean = trans.strip()
            cat_clean = cat.strip()
            
            # Verificar se a transação existe nas originais
            if trans_clean in trans_originais:
                parsed.append((trans_clean, cat_clean))
            else:
                # Tentar match parcial
                for orig_clean, orig_full in trans_originais.items():
                    if trans_clean in orig_clean or orig_clean in trans_clean:
                        parsed.append((orig_clean, cat_clean))
                        break
        elif ':' in line and not line.startswith(('Exemplo', 'Categorias', 'Transações')):
            # Formato alternativo: "Transação: Categoria"
            trans, cat = line.split(':', 1)
            trans_clean = trans.strip()
            cat_clean = cat.strip()
            
            if trans_clean in trans_originais:
                parsed.append((trans_clean, cat_clean))
    
    print(f"✅ Parsed {len(parsed)} categorias de {len(original_transactions)} transações")
    return parsed

# ======== CATEGORIZAÇÃO COM CACHE E BLOCOS ========
def categorize_transactions_in_blocks(df, model_name=OLLAMA_MODEL, bloco_tamanho=BLOCO_TAMANHO):
    """Categoriza transações usando cache e processamento em blocos"""
    
    # Filtrar apenas despesas (valores positivos no seu caso)
    df_despesas = df[df[VALOR_COL] > 0].copy()
    transacoes = df_despesas[DESCRICAO_COL].tolist()
    
    print(f"📊 Total de despesas encontradas: {len(transacoes)}")
    
    cache = load_cache(CACHE_PATH)
    print(f"📁 Cache carregado com {len(cache)} entradas")
    
    llm = ChatOllama(model=model_name)
    
    # Verificar quais transações precisam ser categorizadas
    transacoes_para_categorizar = []
    indices_para_categorizar = []
    
    for idx, t in enumerate(transacoes):
        t_clean = clean_transaction_name(t)
        if t_clean not in cache:
            transacoes_para_categorizar.append(t)
            indices_para_categorizar.append(idx)
    
    print(f"🔄 {len(transacoes_para_categorizar)} transações novas para categorizar com LLM.")
    
    # Processar transações não cachadas por blocos
    if transacoes_para_categorizar:
        for i in tqdm(range(0, len(transacoes_para_categorizar), bloco_tamanho), desc="Categorizando"):
            bloco = transacoes_para_categorizar[i:i+bloco_tamanho]
            prompt = generate_prompt(bloco)
            
            try:
                print(f"\n🤖 Enviando bloco de {len(bloco)} transações para o LLM...")
                resposta = llm.invoke([HumanMessage(content=prompt)])
                resultado = parse_llm_response(resposta.content, bloco)
                
                # Salvar no cache apenas se o parsing foi bem-sucedido
                if len(resultado) > 0:
                    for trans, cat in resultado:
                        t_clean = clean_transaction_name(trans)
                        # Só adiciona ao cache se não existir ou se a categoria anterior era "Outros"
                        if t_clean not in cache or cache[t_clean] == "Outros":
                            cache[t_clean] = cat
                            print(f"  ✅ {t_clean} -> {cat}")
                        else:
                            print(f"  ⏭️ {t_clean} já existe no cache: {cache[t_clean]}")
                else:
                    print(f"  ⚠️ Nenhuma categoria parseada do LLM")
                
            except Exception as e:
                print(f"❌ Erro ao processar bloco: {e}")
                # Em caso de erro, categorizar como "Outros"
                for trans in bloco:
                    t_clean = clean_transaction_name(trans)
                    cache[t_clean] = "Outros"
    
    # Salvar cache atualizado após cada bloco
        if transacoes_para_categorizar:
            save_cache(cache, CACHE_PATH)
        
        print(f"💾 Cache salvo com {len(cache)} entradas")
    
    # Aplicar categorias a todas as transações
    categorias_finais = []
    for t in transacoes:
        t_clean = clean_transaction_name(t)
        categoria = cache.get(t_clean, "Não categorizado")
        categorias_finais.append(categoria)
    
    df_despesas['Categoria'] = categorias_finais
    
    # Debug: mostrar resultado
    print("\n📋 Resumo das categorias:")
    print(df_despesas['Categoria'].value_counts())
    
    return df_despesas

# ======== EXECUÇÃO PRINCIPAL ========
def main():
    """Função principal"""
    print("🚀 Iniciando categorização de transações...")
    
    # Carregar dados
    try:
        df = pd.read_csv(CSV_PATH)
        print(f"📄 Arquivo carregado: {len(df)} transações")
    except FileNotFoundError:
        print(f"❌ Arquivo não encontrado: {CSV_PATH}")
        return
    except Exception as e:
        print(f"❌ Erro ao carregar arquivo: {e}")
        return
    
    # Remover coluna Identificador se existir
    if 'Identificador' in df.columns:
        df.drop(columns=['Identificador'], inplace=True)
        print("🗑️ Coluna 'Identificador' removida")
    
    # Verificar se as colunas necessárias existem
    if DESCRICAO_COL not in df.columns or VALOR_COL not in df.columns:
        print(f"❌ Colunas necessárias não encontradas. Esperado: {DESCRICAO_COL}, {VALOR_COL}")
        print(f"Colunas encontradas: {list(df.columns)}")
        return
    
    # Categorizar transações
    df_categorizado = categorize_transactions_in_blocks(df, model_name=OLLAMA_MODEL, bloco_tamanho=BLOCO_TAMANHO)
    
    # Fazer merge com dados originais (evitando duplicatas)
    df_categorizado_unique = df_categorizado[[DESCRICAO_COL, 'Categoria']].drop_duplicates(subset=[DESCRICAO_COL])
    
    df_final = df.merge(
        df_categorizado_unique, 
        on=DESCRICAO_COL, 
        how='left'
    )
    
    # Preencher categorias vazias (receitas/transferências ficam sem categoria ou com categoria específica)
    df_final['Categoria'] = df_final['Categoria'].fillna('')
    
    # Aplicar lógica específica para valores negativos
    mask_negativos = df_final[VALOR_COL] < 0
    df_final.loc[mask_negativos & df_final['Categoria'].isna(), 'Categoria'] = 'Receita'
    
    # Salvar resultado
    output_path = "extrato_categorizados_final.csv"
    df_final.to_csv(output_path, index=False)
    print(f"✅ Arquivo final salvo como '{output_path}'")
    
    # Mostrar estatísticas finais
    print(f"\n📊 Estatísticas finais:")
    print(f"Total de transações: {len(df_final)}")
    print(f"Despesas categorizadas: {len(df_final[df_final[VALOR_COL] > 0])}")
    print(f"Receitas/Créditos: {len(df_final[df_final[VALOR_COL] <= 0])}")
    
    # Mostrar distribuição de categorias
    categorias_count = df_final[df_final['Categoria'] != '']['Categoria'].value_counts()
    print(f"\n🏷️ Distribuição de categorias:")
    for cat, count in categorias_count.items():
        print(f"  {cat}: {count}")

if __name__ == "__main__":
    main()

🚀 Iniciando categorização de transações...
📄 Arquivo carregado: 49 transações
📊 Total de despesas encontradas: 46
📁 Cache carregado com 0 entradas
🔄 46 transações novas para categorizar com LLM.


Categorizando:   0%|          | 0/5 [00:00<?, ?it/s]


🤖 Enviando bloco de 10 transações para o LLM...


Categorizando:  20%|██        | 1/5 [00:02<00:11,  2.92s/it]

🔍 Resposta do LLM (10 linhas):
  -> Uber* Trip - Transporte
  -> 99app *99app - Transporte
  -> Uber Uber *Trip Help.U - Transporte
  -> Uber* Trip - Transporte
  -> Uber Uber *Trip Help.U - Transporte
✅ Parsed 10 categorias de 10 transações
  ✅ Uber* Trip -> Transporte
  ✅ 99app *99app -> Transporte
  ✅ Uber Uber *Trip Help.U -> Transporte
  ⏭️ Uber* Trip já existe no cache: Transporte
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte
  ⏭️ Uber* Trip já existe no cache: Transporte
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte
  ⏭️ Uber* Trip já existe no cache: Transporte
  ⏭️ 99app *99app já existe no cache: Transporte
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte

🤖 Enviando bloco de 10 transações para o LLM...


...

🔍 Resposta do LLM (10 linhas):
  -> Adega 4d - Alimentação
  -> Uber Uber *Trip Help.U - Transporte
  -> 99app *99app - Outros
  -> 99app *99app - Outros
  -> Uber Uber *Trip Help.U - Transporte
✅ Parsed 10 categorias de 10 transações
  ✅ Adega 4d -> Alimentação
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte
  ⏭️ 99app *99app já existe no cache: Transporte
  ⏭️ 99app *99app já existe no cache: Transporte
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte
  ✅ Gustavo Moura de Sa -> Transferências
  ✅ Gustavo Mattos Fortes -> Transferências
  ⏭️ 99app *99app já existe no cache: Transporte
  ⏭️ 99app *99app já existe no cache: Transporte
  ✅ Parcelamento de Compra - Ppro *Microsoft -> Mercado

🤖 Enviando bloco de 10 transações para o LLM...


Categorizando:  80%|████████  | 4/5 [00:07<00:01,  1.61s/it]

🔍 Resposta do LLM (10 linhas):
  -> Parcelamento de Compra - Nove de Julho Conven - Outros
  -> Nove de Julho Conven - Outros
  -> Blablacar - Transporte
  -> 99app *99app - Mercado
  -> Uber* Trip - Transporte
✅ Parsed 10 categorias de 10 transações
  ✅ Parcelamento de Compra - Nove de Julho Conven -> Outros
  ✅ Nove de Julho Conven -> Outros
  ✅ Blablacar -> Transporte
  ⏭️ 99app *99app já existe no cache: Transporte
  ⏭️ Uber* Trip já existe no cache: Transporte
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte
  ⏭️ 99app *99app já existe no cache: Transporte
  ✅ Prohumana -> Mercado
  ⏭️ 99app *99app já existe no cache: Transporte
  ⏭️ 99app *99app já existe no cache: Transporte

🤖 Enviando bloco de 6 transações para o LLM...


Categorizando: 100%|██████████| 5/5 [00:08<00:00,  1.65s/it]

🔍 Resposta do LLM (6 linhas):
  -> Parcelamento de Compra - Sonda Salto Ii - Mercado
  -> Uber Uber *Trip Help.U - Transporte
  -> Parcelamento de Compra - Sonda Salto Ii - Mercado
  -> Uber* Trip - Transporte
  -> Parcelamento de Compra - Microsoft*Subscription - Educação
✅ Parsed 6 categorias de 6 transações
  ✅ Parcelamento de Compra - Sonda Salto Ii -> Mercado
  ⏭️ Uber Uber *Trip Help.U já existe no cache: Transporte
  ⏭️ Parcelamento de Compra - Sonda Salto Ii já existe no cache: Mercado
  ⏭️ Uber* Trip já existe no cache: Transporte
  ✅ Parcelamento de Compra - Microsoft*Subscription -> Educação
  ✅ Parcelamento de Compra - Fatimo Volmir Oliveir -> Outros
💾 Cache salvo com 18 entradas

📋 Resumo das categorias:
Categoria
Transporte        31
Outros             4
Mercado            4
Alimentação        2
Transferências     2
Lazer              1
Streaming          1
Educação           1
Name: count, dtype: int64
✅ Arquivo final salvo como 'extrato_categorizados_final.csv'

📊 Estat